In [1]:
import os
import numpy as np
import nibabel as nib
from sklearn.cluster import KMeans
from scipy.stats import mode
import glob
from tslearn.clustering import TimeSeriesKMeans
from sklearn.preprocessing import StandardScaler
from scipy.ndimage import gaussian_filter1d

Map created
Map created
Map created
Map created
Map created
Map created
Map created
Map created
Map created
Map created
Map created
Map created
Map created
Map created
Map created
Map created
Map created
Map created
Map created
Map created
Map created
Map created
Map created
Map created
Map created
Map created
Map created
Consensus cluster map saved to /home/fgomezacebo/scratch/consensus_cluster_map_TAQ_plac.nii


In [ ]:
# Parameters
directory_path = "/home/fgomezacebo/scratch" # Directory with patient NIfTI files
n_clusters = 5  # Number of clusters for k-means
nifti_file_paths = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.nii.gz')]

# Initialize a list to store the cluster maps for each patient
patient_cluster_maps = []
nifti_paths = []
specific_string = "ses-plac_filtered_func_data_normalized.nii.gz"
paths = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if specific_string in f]

In [ ]:
for file_path in paths:
    # Load the NIfTI file
    nifti_img = nib.load(file_path)
    data = nifti_img.get_fdata()

    # Identify non-zero voxels
    non_zero_mask = np.any(data != 0, axis=-1)

    # Reshape data to (voxels, time series)
    reshaped_data = data[non_zero_mask].reshape(-1, data.shape[-1])

    # Calculate sum of squared TAC values for each voxel
    sum_squared_TAC = np.sum(np.square(reshaped_data), axis=1)
    
    # Determine the threshold for the top 50% TAC values
    threshold = np.percentile(sum_squared_TAC, 50)

    # Select voxels for clustering (top 50% based on sum of squared TAC)
    selected_voxels = reshaped_data[sum_squared_TAC >= threshold, :]

    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(selected_voxels)
    selected_cluster_labels = kmeans.labels_

    # Get centroids of the clusters
    centroids = kmeans.cluster_centers_

    # Create a full brain map
    full_brain_map = np.full(data.shape[:-1], -1, dtype=int)

    # Assign selected voxels to their clusters
    selected_voxels_3d_indices = np.argwhere(non_zero_mask)[sum_squared_TAC >= threshold]
    for idx_3d, label in zip(selected_voxels_3d_indices, selected_cluster_labels):
        full_brain_map[tuple(idx_3d)] = label

    # Assign remaining voxels to the nearest cluster
    remaining_voxels = reshaped_data[sum_squared_TAC < threshold, :]
    remaining_voxels_3d_indices = np.argwhere(non_zero_mask)[sum_squared_TAC < threshold]
    for idx, voxel in zip(remaining_voxels_3d_indices, remaining_voxels):
        distances = np.linalg.norm(voxel - centroids, axis=1)
        nearest_cluster = np.argmin(distances)
        full_brain_map[tuple(idx)] = nearest_cluster

    print('Map created')


    patient_cluster_maps.append(full_brain_map)

In [ ]:
# Majority voting for consensus cluster map
all_patient_cluster_maps = np.stack(patient_cluster_maps)
consensus_cluster_map = mode(all_patient_cluster_maps, axis=0).mode.squeeze()

# Create and save the consensus cluster map NIfTI image
consensus_cluster_nifti = nib.Nifti1Image(consensus_cluster_map.astype(np.int16), nifti_img.affine)

# Save the consensus cluster map as a new NIfTI file
consensus_map_file_path = os.path.join(directory_path, 'consensus_cluster_map_TAQ_plac.nii')
nib.save(consensus_cluster_nifti, consensus_map_file_path)

print(f"Consensus cluster map saved to {consensus_map_file_path}")